In [1]:
from time import time
import pandas as pd
import numpy as np
from datetime import datetime
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from binance import Client
import time

client = Client()

smooth_interval = 25
smooth_exp = 3
rango = 0
std_mult = 0.25
pdi_mult = 500
sdi_mult = 100000
z_aprox = 0.00005

start = time.time()
recent_data = pd.read_csv('../../../../../csvs/historical/by1m/BTCUSDT')
recent_data = recent_data.iloc[:1000,:6]
recent_data.columns = ['Tiempo','Open','High','Low','Close','Volume']
recent_data['Tiempo'] = recent_data['Tiempo'].astype('datetime64[ns]')# pd.to_datetime(recent_data.Tiempo,unit='ms')
recent_data = recent_data.set_index('Tiempo')
recent_data = recent_data.astype(float)
recent_data = recent_data.reset_index()
end = time.time()
print(f'{end-start}')
recent_data

2.4494309425354004


,Tiempo,Open,High,Low,Close,Volume
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...,...
995,2017-08-17 20:35:00,4302.96,4302.96,4302.96,4302.96,0.052740
996,2017-08-17 20:36:00,4302.96,4302.96,4302.96,4302.96,0.585532
997,2017-08-17 20:37:00,4301.91,4301.91,4301.91,4301.91,0.445848
998,2017-08-17 20:38:00,4301.91,4301.91,4301.91,4301.91,0.372403


In [2]:
start = time.time()
recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Open,smooth_interval,smooth_exp)
recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
# recent_data['Tercer_Derivada'] = recent_data.SD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
# recent_data['TD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Tercer_Derivada,smooth_interval,smooth_exp)
end = time.time()
print("DERIVATIVES = %s" % (end - start))

DERIVATIVES = 0.3417978286743164


In [3]:
def savgol_filter(y, window_size, poly_order):
    half_window = window_size // 2
    order_range = np.arange(poly_order+1)
    weight = np.zeros((window_size, poly_order+1))
    
    for i in range(-half_window, half_window+1):
        weight[i+half_window, :] = [i**j for j in order_range]
    
    weights = np.linalg.pinv(weight).T.sum(axis=0)
    
    smoothed = np.convolve(weights, y, mode='valid')
    
    return smoothed #smoothed_padded

def calculate_derivatives(interpolacion, times, derivative):
    d_interpolacion = np.diff(interpolacion) / times[2+derivative:]
    return d_interpolacion

start = time.time()
open_prices = recent_data['Open']
times = (np.diff(recent_data.Tiempo)/np.timedelta64(1, 's')).astype(float)

interpolacion = savgol_filter(open_prices, smooth_interval, 3)
primer_derivada = calculate_derivatives(interpolacion, times, 1)
pd_interpolacion = savgol_filter(primer_derivada, smooth_interval, 3)
segunda_derivada = calculate_derivatives(pd_interpolacion, times, 5)
sd_interpolacion = savgol_filter(segunda_derivada, smooth_interval, 3)
end = time.time()
print("CPU ON DERIVATIVES = %s" % (end - start))

CPU ON DERIVATIVES = 0.0019681453704833984


In [27]:
recent_data['sdi_updown_copy'] = 0
recent_data['pdi_updown_copy'] = 0
recent_data['sdi_updown'] = 0
recent_data['pdi_updown'] = 0
abs_open_std = abs(np.std(recent_data.Open)*std_mult)


start = time.time()
recent_data['sdi_updown_copy'] = np.select(condlist=[(abs(np.concatenate([np.zeros(11),sd_interpolacion])*sdi_mult) > abs_open_std)],
                                                        choicelist=[1],
                                                        default=0)
recent_data['pdi_updown_copy'] = np.select(condlist=[((np.concatenate([np.zeros(7),pd_interpolacion])*pdi_mult) > abs_open_std),
                                                        ((np.concatenate([np.zeros(7),pd_interpolacion])*pdi_mult) < -abs_open_std)],
                                                        choicelist=[1,-1],
                                                        default=0)
end = time.time()
print(f'{end-start}')

start = time.time()
recent_data['pdi_updown'] = np.select(condlist=[((recent_data.PD_Interpolacion*pdi_mult) > abs_open_std),
                                                        ((recent_data.PD_Interpolacion*pdi_mult) < -abs_open_std)],
                                                        choicelist=[1,-1],
                                                        default=0)
recent_data['sdi_updown'] = np.select(condlist=[(abs(recent_data.SD_Interpolacion*sdi_mult) > abs_open_std)],
                                                        choicelist=[1],
                                                        default=0)
end = time.time()
print(f'{end-start}')

0.0009989738464355469
0.001001119613647461


In [18]:
recent_data[recent_data['sdi_updown'] == recent_data['sdi_updown_copy']]

,Tiempo,Open,High,Low,Close,Volume,Interpolacion,Primer_Derivada,PD_Interpolacion,Segunda_Derivada,SD_Interpolacion,Tercer_Derivada,TD_Interpolacion,sdi_updown,pdi_updown,sell_buy_side,signal,sdi_updown_copy
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,4265.921597,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,4265.254896,-0.011112,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,4264.629019,-0.010431,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,4264.047034,-0.009700,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,4263.512004,-0.008917,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,2017-08-17 20:33:00,4287.08,4302.96,4287.08,4302.96,0.552043,4284.252280,0.020005,0.013452,0.000206,0.000208,7.439504e-07,7.843430e-07,1,0,1,0,1
994,2017-08-17 20:34:00,4302.96,4302.96,4302.96,4302.96,0.042023,4286.370679,0.035307,0.028438,0.000250,0.000255,7.738708e-07,8.234400e-07,1,0,1,0,1
995,2017-08-17 20:35:00,4302.96,4302.96,4302.96,4302.96,0.052740,4289.513983,0.052388,0.046205,0.000296,0.000302,7.968333e-07,8.470619e-07,1,1,1,1,1
997,2017-08-17 20:37:00,4301.91,4301.91,4301.91,4301.91,0.445848,4299.302547,0.091892,0.090698,0.000397,0.000401,8.218845e-07,8.401534e-07,1,1,1,1,1


In [8]:
start = time.time()

max_points_idx = []
min_points_idx = []
pmin_idx = 0
pmax_idx = 0
abs_open_std = abs(np.std(recent_data.Open)*std_mult)

recent_data['sell_buy_side'] = 0
recent_data['sdi_updown'] = 0
recent_data['pdi_updown'] = 0
recent_data['signal'] = 0

recent_data['sdi_updown'] = np.select(condlist=[(abs(recent_data.SD_Interpolacion*sdi_mult) > abs_open_std)],
                                                        choicelist=[1],
                                                        default=0)
recent_data['pdi_updown'] = np.select(condlist=[((recent_data.PD_Interpolacion*pdi_mult) > abs_open_std),
                                                        ((recent_data.PD_Interpolacion*pdi_mult) < -abs_open_std)],
                                                        choicelist=[1,-1],
                                                        default=0)
recent_data['sell_buy_side'] = np.select(condlist=[recent_data.SD_Interpolacion > 0,
                                                        recent_data.SD_Interpolacion < 0],
                                                        choicelist=[1,-1],
                                                        default=0)

recent_data.loc[(recent_data.sell_buy_side == 1) & (recent_data.sdi_updown == 1) & (recent_data.pdi_updown == 1),'signal'] = 1
recent_data.loc[(recent_data.sell_buy_side == -1) & (recent_data.sdi_updown == 1) & (recent_data.pdi_updown == -1),'signal'] = -1

end = time.time()
print(f'{end-start}')

0.004000425338745117
